In [1]:
from utils import load_env_vars
from openai import OpenAI
import time, json, os, tqdm

load_env_vars()

# Initialize OpenAI client
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

In [3]:
from utils_gpt import prepare_training_data

task = "P"

for split in ["train", "val"]:
    prepare_training_data(f"data/lamp_{task}_{split}.json", f"data/lamp_{task}_{split}_gpt_format.jsonl")


In [4]:
training_fn = f"data/lamp_{task}_train_gpt_format.jsonl"
val_fn = f"data/lamp_{task}_val_gpt_format.jsonl"

training_file = client.files.create(file=open(training_fn, 'rb'), purpose='fine-tune')
val_file = client.files.create(file=open(val_fn, 'rb'), purpose='fine-tune')
print(training_file)
print(val_file)

FileObject(id='file-LrT8QT4xZ6PN6o24QGHgeN', bytes=4869084, created_at=1732740114, filename='lamp_P_train_gpt_format.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
FileObject(id='file-JxPKqZ4XoL18mX3FJAchdZ', bytes=291652, created_at=1732740114, filename='lamp_P_val_gpt_format.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


In [6]:
# base_model, suffix = "gpt-4o-mini-2024-07-18", f"lamp-4o-mini-{task}"
base_model, suffix = "gpt-4o-2024-08-06", f"lamp-4o-{task}"

hyperparameters = {"n_epochs": 3}

job = client.fine_tuning.jobs.create(training_file=training_file.id, model=base_model, suffix=suffix, validation_file=val_file.id, hyperparameters=hyperparameters)

In [9]:
from datetime import datetime

while True:
    job_status = client.fine_tuning.jobs.retrieve(job.id)
    if job_status.status == "running":
        if not job_status.estimated_finish:
            print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Status: {job_status.status}, Estimated completion: N/A")
        else:
            estimated_completion = datetime.fromtimestamp(job_status.estimated_finish)
            print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Status: {job_status.status}, Estimated completion: {estimated_completion}")
    else:
        print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Status: {job_status.status}")
    
    if job_status.status in ['succeeded', 'failed']:
        if job_status.status == "succeeded":
            print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Fine-tuning job completed successfully")     
            print(f"Model: {job_status.fine_tuned_model}")       
        else:
            print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Fine-tuning job failed")
        break
        
    time.sleep(60)  # Check status every minute



[2024-11-27 15:47:53] Status: running, Estimated completion: N/A
[2024-11-27 15:48:53] Status: running, Estimated completion: N/A
[2024-11-27 15:49:53] Status: running, Estimated completion: 2024-11-27 16:52:01
[2024-11-27 15:50:54] Status: running, Estimated completion: 2024-11-27 16:51:44
[2024-11-27 15:51:54] Status: running, Estimated completion: 2024-11-27 16:52:45
[2024-11-27 15:52:54] Status: running, Estimated completion: 2024-11-27 16:55:36
[2024-11-27 15:53:55] Status: running, Estimated completion: 2024-11-27 16:55:38
[2024-11-27 15:54:55] Status: running, Estimated completion: 2024-11-27 16:55:06
[2024-11-27 15:55:55] Status: running, Estimated completion: 2024-11-27 16:54:57
[2024-11-27 15:56:56] Status: running, Estimated completion: 2024-11-27 16:55:42
[2024-11-27 15:57:56] Status: running, Estimated completion: 2024-11-27 16:55:49
[2024-11-27 15:58:56] Status: running, Estimated completion: 2024-11-27 16:55:14
[2024-11-27 15:59:56] Status: running, Estimated completion:

In [10]:
job_status

FineTuningJob(id='ftjob-ENZTqiCLaEm4ShW3nDcC24oS', created_at=1732740238, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-2024-08-06:salesforce-research:lamp-4o-p:AYKM53Ac', finished_at=1732744759, hyperparameters=Hyperparameters(n_epochs=3, batch_size=3, learning_rate_multiplier=2), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-Y7zi4Bj4dOxFLW4CpEFnAffn', result_files=['file-BKHNdAZ7b4nr1oLAzfeNbC'], seed=404531127, status='succeeded', trained_tokens=2999766, training_file='file-LrT8QT4xZ6PN6o24QGHgeN', validation_file='file-JxPKqZ4XoL18mX3FJAchdZ', estimated_finish=None, integrations=[], user_provided_suffix='lamp-4o-P')